# 2.11 Matrix free operator application
Usually, we assemble matrices in sparse matrix format. Certain methods allow to improve performance and reduce memory requirements considerably by avoiding building and storing the system matrix. The counterpart of this approach is that it is now difficult to build preconditioners

## Hybrid mixed method

consider the hybrid mixed method

$$
\begin{array}{ccccl}
A \sigma & + & B^T u & = & f \\
B u & & & = & g
\end{array}
$$

where the matrix $A$ comes from

$$
\int \sigma \tau dx 
$$

and the matrix $B$ from

$$
\sum_T \int_T \operatorname{div} \, \sigma \, v_T - \int_{\partial T} \sigma_n v_F
$$

For $\sigma$ we use a `VectorL2` space, with a Piola-mapping to the physical elements

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from ngsolve.fem import MixedFE
from netgen.csg import *

geom = CSGeometry()
geom.Add (Sphere(Pnt(50,50,50),80) \
          -Cylinder(Pnt(-100,0,0),Pnt(200,0,0), 40) \
          -Cylinder(Pnt(100,-100,100),Pnt(100,200,100),40)
          -Cylinder(Pnt(0,100,-100), Pnt(0,100,200),40)
          -Sphere(Pnt(50,50,50),50))
# geom.Draw()

mesh = Mesh(geom.GenerateMesh(maxh=25))

#for i in range(1):
#    mesh.Refine()
#mesh.Curve(5)
# Draw (mesh)

Same as above, with the `geom_free` flag for the `BilinearForm`:

In [ ]:
order=5
SetHeapSize(100*1000*1000)
Sigma = VectorL2(mesh, order=order, piola=True)
Vt = L2(mesh, order=order-1)
Vf = FacetFESpace(mesh, order=order, dirichlet=[2])
V = Vt*Vf
print ("Sigma.ndof =", Sigma.ndof, ", V.ndof =", V.ndof)
sigma,tau = Sigma.TnT()
(ut,uf), (vt,vf) = V.TnT()

b = BilinearForm(trialspace=Sigma, testspace=V, geom_free=False)
b += div(sigma) * vt * dx
n = specialcf.normal(mesh.dim)
dS = dx(element_boundary=True)
b += -sigma*n*vf * dS
b.Assemble()

`L2` finite element spaces have a `Mass` method generating a linear operator for multiplication with the diagonal mass matrix. In case of variable coefficients or curved elements it uses numerical integration. It also provide an inverse operator:

In [ ]:
from ngsolve.ngscuda import *
#ainv = Sigma.Mass(rho=1).Inverse()
ainv = BilinearForm(sigma*tau*dx, diagonal=True).Assemble().mat.Inverse()

Combine the operators to form the Schur complement, which is actually a discretization of the Laplace operator:

In [ ]:
Laplace = b.mat @ ainv @ b.mat.T

f = LinearForm (V)
f += 1*vt * dx
f.Assemble()

proj = Projector(V.FreeDofs(), True)

gfu = GridFunction (V)

The iteration count is very high since we used only the most trivial preconditioner. We can add a low order coarse-grid preconditioner to get rid of the mesh dependency, and also local inverses to improve the $p$-dependency.

In [ ]:
import time

#ngsglobals.msg_level = 9

adev = Laplace.CreateDeviceMatrix()
projdev = proj.CreateDeviceMatrix()

fdev = UnifiedVector(f.vec)
gfudev = UnifiedVector(gfu.vec)

t = time.time()
with TaskManager():
  solvers.CG(mat=Laplace, pre=proj, rhs=f.vec, sol=gfu.vec, maxsteps=5000, printrates=False)
print(Norm(gfu.vec))
print(time.time() - t)

t = time.time()
solvers.CG(mat=adev, pre=projdev, rhs=fdev, sol=gfudev, maxsteps=5000, printrates=False)
print(Norm(gfudev))
print(time.time() - t)